## Imports

* [NiPy](https://nipy.org/)
* [NiLearn](https://nilearn.github.io/)





In [ ]:
# install on first run

!pip install nilearn
!pip install dicom2nifti # if src file in dcm, not nii
!pip install pathlib

  Using cached dicom2nifti-2.3.0.tar.gz (33 kB)
  Using cached pydicom-2.2.1-py3-none-any.whl (2.0 MB)
  Created wheel for dicom2nifti: filename=dicom2nifti-2.3.0-py3-none-any.whl size=42932 sha256=ad94dcb15364839fe46706cf598e5091f1aceed9edbd1b1070785ffd8cbaf031
  Stored in directory: /root/.cache/pip/wheels/bf/ff/aa/f5b52fd2e9f9355a28a6f6f6994d721a13209387528b146db5
Successfully built dicom2nifti


In [ ]:
# imports 

from nilearn import plotting
from nilearn import image
import nibabel as nb

import pydicom as dicom
import dicom2nifti #if src file in dcm
import matplotlib
import pylab as plt
import matplotlib.patches
from PIL import Image
import numpy as np
import pandas as pd
import scipy.ndimage as ndi
import scipy.stats as stats
from datetime import *

import os
from glob import glob
from pathlib import Path
# from collections import defaultdict
# import pickle
# import sys

from IPython.display import Markdown, display
from IPython.display import Image

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Mounting

In [ ]:
# Mount Google Drive

from google.colab import drive

drive.mount('gdrive/')
# Directions: 
#  1) run cell, 2) click link, 3) choose acount & sign in, 4) copy authorization code, 5) Paste & Enter, 6) wait patiently.

Mounted at gdrive/


## Pathing

In [ ]:
src = '/content/gdrive/My Drive/SNUBH/RodNiPy/data/PETandCT/'
pth = src + '2020/' + '20201111_BA-1907-276-052-05_F-18-FDG_LHY/' + '1/'
ct_pth = pth + 'CT'
pet_pth = pth + 'PET'
ans_pth = pth + 'Answer/'
wb_name = '20201111_WB_No1_CT.nii'
wb_path = ans_pth + wb_name
Path(pth + 'Results/').mkdir(parents=True, exist_ok=True)  # create dir if nonexistant
rslt_pth = os.path.join(pth, 'Results')
wb_path

# CT

# Head Segmentation

In [ ]:
head_arr = wb_arr[210:-210, wb_z_y_range[0]:wb_z_y_range[-1], 330:-35]
head_arr.shape

In [ ]:
# niify
head_img = nb.Nifti1Image(head_arr, affine=np.eye(4))

In [ ]:
# visualize
# WB-segmented mid-point y-coord

plotting.plot_matrix(wb_arr[:, int(np.median(wb_z_y_range)), :])

In [ ]:
# visualize
# WB-segmented mid-point y-coord
plotting.plot_matrix(head_arr[:, 35, :])

In [ ]:
# # Visualization head_arr
#
plotting.view_img(head_img, bg_img=False, title="head_img", cmap='cold_hot')   

In [ ]:
# # Visualization wb_arr
#
plotting.view_img(wb_img, bg_img=False, title="wb_arr", cmap='cold_hot')  

In [ ]:
# Saver

print('Save? (Y/N): ')
inp = input().capitalize()
if inp =='Y':
  hd_name = wb_name.replace('WB', 'HD')
  nb.save(head_img, os.path.join(rslt_pth, hd_name) )
  print(hd_name, 'saved in ', rslt_pth)

In [ ]:
# find max abs z-score slice

max_z = 0
max_z_i = 0

for i, value in enumerate(head_y_sums_z):
  if abs(value) > max_z:
    max_z = abs(value)
    max_z_i = i

#max_z_i = wb_z_y_range[max_z_i]
print(max_z_i, max_z, head_y_sums_z[max_z_i]) #test
print(wb_z_y_range[max_z_i])

In [ ]:
# find max

# initalize
max_head = 0
max_labels = np.zeros(wb_arr.shape)
prev_sum = 0
lab_ind = -1

for i in wb_z_y_range:  
  if head_mask_sum != prev_sum: #filter for non-sames
    prev_sum = head_mask_sum

    if head_mask_sum > max_head:
      max_head = head_mask_sum
      max_labels = labels
      lab_ind = head_label

print('max_head', max_head)

#mask
true_mask = np.where(max_labels == lab_ind, 1 ,np.nan).astype(int)
print('true_mask shape', true_mask.shape) 

In [ ]:
# create 3D box

# 3D box label
loc = ndi.find_objects(true_mask)[0] # x, z coord of true_mask
print('loc:', loc)

x1 = loc[0].start-2 if loc[0].start >= 2 else loc[0].start
x2 = loc[0].stop+2 if loc[0].stop+2 < wb_arr.shape[0] else loc[0].stop
y1 = y_list[0]
y2 = y_list[-1]
z1 = loc[1].start-2 if loc[1].start >= 2 else loc[1].start
z2 = loc[1].stop+2 if loc[1].stop+2 < wb_arr.shape[2] else loc[1].stop


#apply box to all slices
_3d_mask = np.zeros([x2-x1, y2-y1, z2-z1])
print('initial _3d_mask shp:', _3d_mask.shape)
print('y2 - y1:', y2 - y1)
print('np.arange(y1,y2).shape:', np.arange(y1,y2) )
cnt = 0

for i in np.arange(y1,y2): 
  print('wb_arr[x1:x2, i, z1:z2].shp', wb_arr[x1:x2, i, z1:z2].shape)
  _3d_mask[:, i,:] = wb_arr[x1:x2, i, z1:z2]
  cnt += 1
print('cnt', cnt)

print('post _3d_mask shp:', _3d_mask.shape)

ni_br = nb.Nifti1Image(_3d_mask, wb_img.affine)
print("ni_br.shape", ni_br.shape)

In [ ]:
### Head Segmentation Function1 ###
# Method1: Find head blob of each Y slice, pick the largest head blob slice, 
#          then create manual 3D box around the slice's central point
# format: [x, y, z]

def headSeg(wb_path):
  wb_img_og = nb.load(wb_path)
  #print('pre-squeeze wb_img_og.shape =', wb_img_og.shape) #TEST
  wb_img = nb.squeeze_image(wb_img_og) # squeeze to 3D
  #print('post-squeeze wb_img.shape =', wb_img.shape) #TEST
  wb_arr = np.array(wb_img.dataobj)

  #find best ref slice 
  num_slices = wb_arr.shape[1] # y-axis

  #initalize best ref finder vars
  max_head = 0
  max_labels = np.zeros(wb_arr.shape)
  lab_ind = -1
  y_list = []
  prev_sum = 0

  #find best reference slice, i.e the largest brain slice
  for i in range(num_slices):  
    curr_slice = wb_arr[:,i,:]

    mask_thresh = np.where(curr_slice > 300, 1,0) #300 is brightness
    mask = ndi.binary_closing(mask_thresh)

    labels, nlabels = ndi.label(mask)
    head_val = labels[10:30,100:140]  ############## HC approx. head loc
    head_label = np.max(head_val)
    head_mask = np.where(labels == head_label,1,0)  
    head_mask_sum = np.sum(head_mask)

    if head_mask_sum != prev_sum and head_mask_sum != 223166: #filter for non-sames
      y_list.append(i)
      prev_sum = head_mask_sum

      if head_mask_sum > max_head:
        max_head = head_mask_sum
        max_labels = labels
        lab_ind = head_label

  print('y_list len', len(y_list))
  print('max_head', max_head)

  #mask
  true_mask = np.where(max_labels == lab_ind, 1 ,np.nan).astype(int)

  # 3D box label
  loc = ndi.find_objects(true_mask)[0] # x, z coord of true_mask

  x1 = loc[0].start-2 if loc[0].start >= 2 else loc[0].start
  x2 = loc[0].stop+2 if loc[0].stop+2 < wb_arr.shape[0] else loc[0].stop
  y1 = y_list[0]
  y2 = y_list[-1]
  z1 = loc[1].start-2 if loc[1].start >= 2 else loc[1].start
  z2 = loc[1].stop+2 if loc[1].stop+2 < wb_arr.shape[2] else loc[1].stop


  #apply box to all slices
  _3d_mask = np.zeros([x2-x1, y2-y1, z2-z1])
  cnt = 0

  for i in np.arange(y1,y2): 
    print('wb_arr[x1:x2, i, z1:z2].shp', wb_arr[x1:x2, i, z1:z2].shape)
    _3d_mask[:, i,:] = wb_arr[x1:x2, i, z1:z2]
    cnt += 1
  print('cnt', cnt)

  print('post _3d_mask shp:', _3d_mask.shape)

  ni_br = nb.Nifti1Image(_3d_mask, wb_img.affine)
  print("ni_br.shape", ni_br.shape)
      
  #  return ni_br

In [ ]:
max_head_nii = nb.Nifti1Image(true_mask, wb_img.affine)
max_head_nii.shape

In [ ]:
# # call headSeg() # to be iterated on all mice
# ni_br = headSeg(wb_path)
# ni_br

In [ ]:
# Saver

print('Save? (Y/N): ')
inp = input().capitalize()
if inp =='Y':
  br_name = wb_name.replace('WB', 'BR')
  nb.save(ni_br, rslt_pth + br_name)
  print(br_name + 'saved in ' + rslt_pth)

In [ ]:
# # 3D Visualization

# # WB plot
# print(wb_img.shape)
# plotting.plot_stat_map(wb_img_og, bg_img=None, colorbar=False, cmap='bone', black_bg=True)  

# # BR plot
# plotting.plot_stat_map(nb.load(rslt_pth + br_name), bg_img=None, colorbar=False, cmap='bone', black_bg=True)